# Lesson 5 - Multiple Agents using CrewAI
Now instead of having one agent, we can add multiple agents to our crew.

In [29]:
# We first install the packages
%pip install crewai langchain langchain-openai --quiet

Note: you may need to restart the kernel to use updated packages.


In [30]:
# We install our environment variables helper library
%pip install python-dotenv --quiet
import dotenv
dotenv.load_dotenv()

Note: you may need to restart the kernel to use updated packages.


True

In [31]:
# We load our helper callback to see what goes over the wire
from _callbacks import PrettyPrintCallbackHandler
callback = PrettyPrintCallbackHandler()

In [32]:
# We load the llm mode using our helper and based onthe settings
import _models
import _settings
llm=_models.get_llm(_settings.MODEL_NAME, callbacks=[callback])


In [33]:
# We load our challenge from the settings
import _settings
challenge_prompt=_settings.CHALLENGE_PROMPT

In [34]:
from crewai import Agent, Task, Crew, Process
import _agents
developer= _agents.get_agent_by_name("senior developer",llm)
ops = _agents.get_agent_by_name("senior devops",llm)

Creating senior developer
Creating senior devops


In [35]:
# Create tasks for your agents
task1 = Task(
  description=challenge_prompt,
  expected_output="A script shown as markdown code block",
  agent=developer,
  llm=llm
)

In [36]:
# Create tasks for your agents
task2 = Task(
  description="""Analyze a shell script and make sure it's compatible with most of the Linux systems.""",
  expected_output="A script shown as markdown code block",
  agent=ops,
  llm=llm
)

In [37]:
# Instantiate your crew with a sequential process
# We have assign specific agents to specific tasks and have the developer kick it off
# In the output we should see them working on the tasks in sequence
crew = Crew(
  agents=[developer],
  tasks=[task1,task2],
  verbose=2, # You can set it to 1 or 2 to different logging levels
  full_output=True,
)

# Get your crew to work!
result = crew.kickoff()

2024-06-15 15:23:42,121 - 8387709952 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Senior Developer
 [INFO]: == Starting Task: Write me a shell script to list the files in the current directory. 
Output the script as markdown code block


> Entering new CrewAgentExecutor chain...
[chat_model][start] - prompts : You are Senior Developer. You have more then with 10 years of experience. 
        You have worked on a variety of projects and have a deep understanding of software development best practices. 
        You are passionate about writing clean, efficient code and am always looking for ways to improve my skills.
        
Your personal goal is: Develop code that is high quality, scalable, and maintainable.To give my best complete final answer to the task use the exact following format:

Thought: I now can give a great answer
Final Answer: my best complete final answer to the task.
Your final answer must be the great and the most complete as possible, it must be outcome described.

I MUST use these formats, my job depends on it!
Current 

In [38]:
# Print the results
print("######################")
print(result['final_output'])

######################
{'final_output': '```bash\n#!/bin/bash\n# This script lists all the files in the current directory\n\n# Check if the user has read permissions in the current directory\nif [ ! -r . ]; then\n    echo "Error: You do not have read permissions in the current directory."\n    exit 1\nfi\n\nls\n``` \n\nTo run this script, save it to a file, give it execute permissions with `chmod +x filename`, and then run it with `./filename`. It will list all the files in the directory where it is run, or print an error message if the user does not have read permissions in the current directory.', 'tasks_outputs': [TaskOutput(description='Write me a shell script to list the files in the current directory. \nOutput the script as markdown code block', summary='Write me a shell script to list the files in...', exported_output='Here is the shell script encapsulated in a markdown code block:\n\n```bash\n#!/bin/bash\n# This script lists all the files in the current directory\n\nls\n```\n\n

In [39]:
# Now we also allow the agents to decide which one delegates the task to the other
# We have assign specific agents to specific tasks and have the developer kick it off
# In the output we should see them working on the tasks in sequence
equal_crew = Crew(
  agents=[developer, ops],
  tasks=[task1,task2],
  verbose=2, # You can set it to 1 or 2 to different logging levels
  full_output=True,
)

# Get your crew to work!
result = equal_crew.kickoff()


2024-06-15 15:23:58,453 - 8387709952 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Senior Developer
 [INFO]: == Starting Task: Write me a shell script to list the files in the current directory. 
Output the script as markdown code block


> Entering new CrewAgentExecutor chain...
[chat_model][start] - prompts : You are Senior Developer. You have more then with 10 years of experience. 
        You have worked on a variety of projects and have a deep understanding of software development best practices. 
        You are passionate about writing clean, efficient code and am always looking for ways to improve my skills.
        
Your personal goal is: Develop code that is high quality, scalable, and maintainable.
You ONLY have access to the following tools, and should NEVER make up tools that are not listed here:

Delegate work to co-worker: Delegate work to co-worker(task: str, context: str, coworker: Optional[str] = None, **kwargs) - Delegate a specific task to one of the following co-workers: [Senior DevOps Engineer]
The input to this tool 

In [ ]:
# Print the results
print("######################")
print(result['final_output'])